## 0. 필요 라이브러리 

In [14]:
import bs4
from bs4 import BeautifulSoup 
import requests
import urllib
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm
import json
import re
import warnings

import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver
import urllib
from urllib.request import urlopen 
driver = webdriver.Chrome()
warnings.filterwarnings(action='ignore')

## 1. 교보문고 크롤링
- 페이지 검사 > 네트워크에서 해당 페이지의 순위 정보에 대한 json에 바로 접근할 수 있는 링크가 존재하였습니다.
- 따라서,for문으로 1부터 20까지 페이지 넘버를 넣어 json 파일 받을 수 있는 방식 채택하였습니다.
- 소제목의 경우, 제목 옆 괄호 안의 문장을 소제목으로 상정하고 크롤링을 진행하였습니다.
- 이벤트 정보의 경우, 세부 페이지의 '이벤트' 항목의 내용을 크롤링하였습니다.
- 이벤트의 경우, bs4를 사용했을 때는 크롤링이 되지 않아 selenium을 활용하여 진행했습니다.
- 추가정보의 경우, 해당 책에 대한 소개글을 기입하였습니다.

In [15]:
rankings = pd.DataFrame(columns =['순위','제목','소제목','상세보기 링크','이벤트 정보','저자','출판사','출판일자','가격','추가정보'] )

In [16]:
url = 'https://product.kyobobook.co.kr/api/gw/pub/pdt/best-seller/total?page=%s&per=20&period=002&bsslBksClstCode=A'

for i in tqdm(range(1,11, 1)):
    source = requests.get(url%str(i))
    #print(source.text)
    json_ob = json.loads(source.text)
    json_ob = json_ob['data']
    best = json_ob['bestSeller']
    
    for k in tqdm(range(len(best))):
        # 제목과 소제목
        if '(' in best[k]['cmdtName']:
            sub_title = best[k]['cmdtName'].split('(')
            title = sub_title[0]
            sub_title = sub_title[1][:-1]
        else:
            sub_title = np.NAN
            title = best[k]['cmdtName']
        
        # 이벤트 정보 크롤링 - 세부 페이지 접속
        event_list = []
        try: 
            driver.get("https://product.kyobobook.co.kr/detail/"+best[k]['saleCmdtid'])
            for i in range(1,5,1):
                try:
                    event = driver.find_element('xpath','//*[@id="scrollSpyProdEvent"]/div[1]/div[2]/ul/li[%s]/div/a/div[2]/div[1]'%str(i)).text
                    event_list.append(event)
                except:
                    break
        except:
            event_list = []

        # 행 추가
        row = {'순위':best[k]['prstRnkn'],
                '제목' :title,
                '소제목' : sub_title,
                '상세보기 링크': "https://product.kyobobook.co.kr/detail/"+best[k]['saleCmdtid'],
                '이벤트 정보' : event_list,
                '저자' : best[k]['chrcName'],
                '출판사' : best[k]['pbcmName'],
                '출판일자' : best[k]['rlseDate'],
                '가격' : int(best[k]['price']),
                '추가정보' : best[k]['inbukCntt']}
        rankings = rankings.append(row, ignore_index= True)

100%|██████████| 10/10 [30:27<00:00, 182.79s/it]


In [18]:
rankings

,순위,제목,소제목,상세보기 링크,이벤트 정보,저자,출판사,출판일자,가격,추가정보
0,1,트렌드 코리아 2023,NaN,https://product.kyobobook.co.kr/detail/S000061...,"[2023을 읽다! 트렌드/전망서, 쇼호스트와 함께하는 LIVE 책수다, 이달의 책...",김난도 외,미래의창,20221005,19000,"세계화의 종말, 갈등과 분열, 그리고 전쟁. 수십 년간 이어져 온 평화와 공존의 시..."
1,2,사랑은 그렇게 하는 것이 아니다,NaN,https://product.kyobobook.co.kr/detail/S000200...,[이달의 에세이 X 후드 망토 담요],김달,빅피시,20221019,16500,"이미 세 권의 책을 베스트셀러로 만든 작가이자, 80만 명의 구독자를 보유한 크리에..."
2,3,하얼빈,NaN,https://product.kyobobook.co.kr/detail/S000061...,"[《클로버》 샘플북 이벤트, 10월 추천도서와 특/별/선/물]",김훈,문학동네,20220803,16000,‘우리 시대 최고의 문장가’ ‘작가들의 작가’로 일컬어지는 소설가 김훈의 신작 장편...
3,4,아버지의 해방일지,NaN,https://product.kyobobook.co.kr/detail/S000061...,[《클로버》 샘플북 이벤트],정지아,창비,20220902,15000,김유정문학상 심훈문학대상 이효석문학상 등을 수상하며 문학성을 두루 입증받은 ‘리얼리...
4,5,불편한 편의점,40만부 기념 벚꽃 에디션,https://product.kyobobook.co.kr/detail/S000001...,[《클로버》 샘플북 이벤트],김호연,나무옆의자,20210420,14000,"누적 판매 40만부 돌파, 2022년 가장 사랑받는 소설\n★★★전 서점 종합베스트..."
...,...,...,...,...,...,...,...,...,...,...
195,195,귀멸학원 1,NaN,https://product.kyobobook.co.kr/detail/S000061...,[만화/라노벨 무료배송 TOP300 차트],Gotouge Koyoharu 외,학산문화사,20220925,6000,"〈귀멸의 칼날〉 공식 스핀오프 만화.\n학생이 된 탄지로와 친구들,\n그리고 주(柱..."
196,197,소설 보다: 가을 2022,NaN,https://product.kyobobook.co.kr/detail/S000061...,[],김기태 외,문학과지성사,20220916,3500,"독자에게 늘 기대 이상의 가치를 전하는 특별 기획, 『소설 보다: 가을 2022』가..."
197,197,미드나잇 라이브러리,NaN,https://product.kyobobook.co.kr/detail/S000001...,[2022년 2학기 : 한 학기 한 권 읽기],매트 헤이그 외,인플루엔셜,20210428,15800,"★★★★★국내 주요 서점 종합 베스트셀러 1위, 2021년 올해의 책 선정\n★★★..."
198,197,벌거벗은 세계사: 전쟁편,NaN,https://product.kyobobook.co.kr/detail/S000061...,"[벌거벗은 한국사 : 사은품 이벤트!, 벌거벗은 세계사 : 전쟁편 출간 이벤트, 1...",tvn<벌거벗은세계사>제작팀,교보문고,20220727,23000,"이 세상에 그냥 일어나는 일은 아무것도 없습니다. 우리 눈에 보이든, 보이지 않든 ..."


## 2. 공공데이터포털 표준데이터셋 메타데이터 수집
- 상위 목록에서 세부페이지 메타데이터를 받고, 그 다음 세부 페이지 링크별로 접속하여 데이터명과 설명글을 크롤링하는 방식을 택하였습니다.

### 2-1. 상위목록 메타데이터 크롤링
- 상위 목록에서는 데이터의 링크, 조회수, 수정일, 분류체계를 크롤링 했습니다.
- 목록 페이지에서 개별 세부 페이지의 href를 모두 크롤링하여 세부페이지 링크 리스트를 따로 만들었습니다.

In [17]:
url2 = "https://www.data.go.kr/tcs/dss/selectDataSetList.do?dType=STD&keyword=&detailKeyword=&publicDataPk=&recmSe=&detailText=&relatedKeyword=&commaNotInData=&commaAndData=&commaOrData=&must_not=&tabId=&dataSetCoreTf=&coreDataNm=&sort=updtDt&relRadio=&orgFullName=&orgFilter=&org=&orgSearch=&currentPage=%s&perPage=10&brm=&instt=&svcType=&kwrdArray=&extsn=&coreDataNmArray=&pblonsipScopeCode="
links= pd.DataFrame(columns = ['데이터링크','조회수','수정일','분류체계'])
acc_links = []
views = []
for i in tqdm(range(1,16,1)):
    html = requests.get(url2%str(i)).text
    soup = BeautifulSoup(html, 'html.parser')
    li = soup.find('div','result-list').find_all('li')
    for k in range(len(li)):
        acc = li[k].a['href']
        acc = 'http://data.go.kr' + acc
        acc_links.append(acc)
        
        view = li[k].find_all('span','data')
        view = view[1]
        view = re.sub('(<([^>]+)>)','', str(view))
        view = view.strip()

        date = li[k].find_all('span','data')
        date = date[0]
        date = re.sub('(<([^>]+)>)','', str(date))
        date = date.strip()

        class_d = li[k].find('p','tag-area')
        class_d = class_d.find('span','labelset brown')
        class_d = re.sub('(<([^>]+)>)','', str(class_d))
        class_d = class_d.strip()

        
        links = links.append({'데이터링크':acc, '조회수':view, '수정일':date, '분류체계':class_d}, ignore_index = True)

100%|██████████| 15/15 [00:26<00:00,  1.78s/it]


In [19]:
links

,데이터링크,조회수,수정일,분류체계
0,http://data.go.kr/data/15013111/standard.do,14919,2022-10-27,문화관광
1,http://data.go.kr/data/15021145/standard.do,15707,2022-10-27,문화관광
2,http://data.go.kr/data/15013107/standard.do,23024,2022-10-27,사회복지
3,http://data.go.kr/data/15017326/standard.do,15313,2022-10-27,공공행정
4,http://data.go.kr/data/15013110/standard.do,20685,2022-10-27,교육
...,...,...,...,...
142,http://data.go.kr/data/15013105/standard.do,12182,2022-05-31,문화관광
143,http://data.go.kr/data/15012893/standard.do,9808,2022-05-31,공공행정
144,http://data.go.kr/data/15012892/standard.do,33907,2022-05-31,공공행정
145,http://data.go.kr/data/15013108/standard.do,11336,2022-05-19,사회복지


### 2-2. 세부 페이지 크롤링
- 세부 페이지에서는 데이터명과 상위 목록에서는 내용이 생략되었던 설명글을 크롤링했습니다. 
- 앞선 단계에서 크롤링한 세부 페이지 링크들을 하나씩 request하여 크롤링하는 방식을 택하였습니다.

In [20]:
conts = pd.DataFrame(columns = ['데이터명', '설명'])
n = 0
for i in tqdm(range(len(acc_links))):
    # 데이터명, 설명 크롤링
    url = acc_links[i]
    
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')

    
    title = soup.find('p', "tit standard-data-title")
    title = re.sub('(<([^>]+)>)', '', str(title))

    cont = soup.find('div', 'cont')
    cont = re.sub('(<([^>]+)>)', '', str(cont))
    cont = cont.strip()
    
    conts = conts.append({'데이터명':title, '설명':cont},ignore_index = True)
    


100%|██████████| 147/147 [05:37<00:00,  2.30s/it]


In [24]:
conts #데이터명 및 설명

,데이터명,설명
0,전국휴양림표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공..."
1,전국가로수길정보표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공..."
2,전국무료급식소표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공..."
3,전국마을기업표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공..."
4,전국평생학습강좌표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공..."
...,...,...
142,전국민박펜션업소표준데이터,"숙박시설을 운영하며, 자연,문화 체험관광에 적합한 시설을 갖추어 제공하는 업소정보 ..."
143,전국무인민원발급정보표준데이터,"""민원처리에 관한 법률"" 등에 따라 행정기관 또는 공공장소 등에 설치하여 민원인이 ..."
144,전국공중화장실표준데이터,"- 공공데이터 제공 표준 기준, 지자체에서 관리하는 공중화장실정보 화장실명, 주소,..."
145,전국어린이집표준데이터,정부가 보유하고 있는 어린이집 주요 정보를 투명하게 공개함으로써 부모들이 보다 쉽고...


### 2-3. 데이터 합치기

In [22]:
public_std_meta = pd.concat([conts,links], axis=1)

In [25]:
public_std_meta

,데이터명,설명,데이터링크,조회수,수정일,분류체계
0,전국휴양림표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공...",http://data.go.kr/data/15013111/standard.do,14919,2022-10-27,문화관광
1,전국가로수길정보표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공...",http://data.go.kr/data/15021145/standard.do,15707,2022-10-27,문화관광
2,전국무료급식소표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공...",http://data.go.kr/data/15013107/standard.do,23024,2022-10-27,사회복지
3,전국마을기업표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공...",http://data.go.kr/data/15017326/standard.do,15313,2022-10-27,공공행정
4,전국평생학습강좌표준데이터,"공공데이터 제공 표준데이터 속성정보(허용값, 표현형식/단위 등)는 [공공데이터 제공...",http://data.go.kr/data/15013110/standard.do,20685,2022-10-27,교육
...,...,...,...,...,...,...
142,전국민박펜션업소표준데이터,"숙박시설을 운영하며, 자연,문화 체험관광에 적합한 시설을 갖추어 제공하는 업소정보 ...",http://data.go.kr/data/15013105/standard.do,12182,2022-05-31,문화관광
143,전국무인민원발급정보표준데이터,"""민원처리에 관한 법률"" 등에 따라 행정기관 또는 공공장소 등에 설치하여 민원인이 ...",http://data.go.kr/data/15012893/standard.do,9808,2022-05-31,공공행정
144,전국공중화장실표준데이터,"- 공공데이터 제공 표준 기준, 지자체에서 관리하는 공중화장실정보 화장실명, 주소,...",http://data.go.kr/data/15012892/standard.do,33907,2022-05-31,공공행정
145,전국어린이집표준데이터,정부가 보유하고 있는 어린이집 주요 정보를 투명하게 공개함으로써 부모들이 보다 쉽고...,http://data.go.kr/data/15013108/standard.do,11336,2022-05-19,사회복지
